In [1]:
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pandas_gbq
import glob
from google.cloud import bigquery
import re

In [2]:
client = bigquery.Client()

In [12]:
query = """ 
        SELECT * FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2024_fev_militares_reserva_reforma_cadastro` 
        """  

In [14]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,COD_TIPO_APOSENTADORIA,TIPO_APOSENTADORIA,DATA_APOSENTADORIA,DESCRICAO_CARGO,COD_UORG_LOTACAO,UORG_LOTACAO,...,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,ANO,MES
0,3021564,SIDNEY GOMES PEREIRA,***.033.690-**,000****,3,REFORMA POR INVALIDEZ,07/10/1952,Segundo-Sargento,-1,Sem informação,...,01/01/1952,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2023,3
1,667339,JOSE REYNALDO SOUZA GOES,***.675.285-**,000****,3,REFORMA POR INVALIDEZ,26/02/1993,Segundo-Tenente,-1,Sem informação,...,01/01/1957,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2023,3
2,3612242,RENE GREBOGE DE ALMEIDA,***.895.709-**,000****,2,REFORMA,19/05/1975,Cabo (engajado),-1,Sem informação,...,01/01/1966,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2023,3
3,2980660,DOURIVAL TEIXEIRA DOS SANTOS,***.222.985-**,000****,3,REFORMA POR INVALIDEZ,22/11/1972,Terceiro-Sargento,-1,Sem informação,...,01/01/1968,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2023,3
4,3127285,LUIZ CARLOS DO CARMO,***.328.899-**,000****,2,REFORMA,21/02/2006,Segundo-Tenente,-1,Sem informação,...,01/01/1968,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2023,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168104,734617,PAULO ROBERTO DOS SANTOS,***.037.546-**,099****,2,REFORMA,27/02/2019,Capitão-de-Fragata,-1,Sem informação,...,07/03/1979,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2023,3
168105,3454011,RICARDO LUTERMAN,***.049.988-**,099****,2,REFORMA,24/02/2017,Capitão-de-Fragata,-1,Sem informação,...,07/03/1979,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2023,3
168106,2439438,ROBERTO SIMAS CORDEIRO,***.705.257-**,099****,5,REFORMA POR DOENÇA,12/11/2004,Capitão-de-Mar-e-Guerra,-1,Sem informação,...,07/03/1979,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2023,3
168107,3373745,RONALDO NETTO PIRES,***.833.237-**,099****,2,REFORMA,19/02/2016,Capitão-de-Fragata,-1,Sem informação,...,07/03/1979,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2023,3


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168109 entries, 0 to 168108
Data columns (total 29 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Id_SERVIDOR_PORTAL                    168109 non-null  Int64  
 1   NOME                                  168109 non-null  object 
 2   CPF                                   168109 non-null  object 
 3   MATRICULA                             168109 non-null  object 
 4   COD_TIPO_APOSENTADORIA                168109 non-null  Int64  
 5   TIPO_APOSENTADORIA                    168109 non-null  object 
 6   DATA_APOSENTADORIA                    168090 non-null  object 
 7   DESCRICAO_CARGO                       168109 non-null  object 
 8   COD_UORG_LOTACAO                      168109 non-null  Int64  
 9   UORG_LOTACAO                          168109 non-null  object 
 10  COD_ORG_LOTACAO                       168109 non-null  Int64  
 11  

In [17]:
df['DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO'].unique()

array([nan])

DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO - DATA_INICIO_PENSAO ?       


In [53]:
df['tipo_pensao'].unique()

array(['Não informado', 'Cônjuge / Viúva (o)', 'Companheiro (a)',
       'Pessoa desquitada, separada judicialmente, divorciada do instituidor ou ex-convivente',
       'Filha', 'Filho', 'Mãe', 'Pai', 'Neto (a)', 'Irmão (ã)',
       'Menor sob guarda ou tutela',
       'Pessoa designada (Beneficiário instituído)',
       'Ex-combatente (o próprio)',
       'Outros (Pessoas sem Vínculo Militar)',
       'Filho(a) adotivo ou Enteado(a)'], dtype=object)

In [54]:
pensoes = {
    "Filho" : "Filhos, netos, enteados ou menores em tutela",
    "Filha" : "Filhos, netos, enteados ou menores em tutela",
    "Menor sob guarda ou tutela" : "Filhos, netos, enteados ou menores em tutela",
    "Neto (a)" : "Filhos, netos, enteados ou menores em tutela",
    "Filho(a) adotivo ou Enteado(a)" : "Filhos, netos, enteados ou menores em tutela",
    "Cônjuge / Viúva (o)" : "Cônjuge ou ex-cônjuge",
    "Companheiro (a)" : "Cônjuge ou ex-cônjuge",
    "Pessoa desquitada, separada judicialmente, divorciada do instituidor ou ex-convivente" : "Cônjuge ou ex-cônjuge",
    "Mãe":"Pais",
    "Pai":"Pais",
    "Irmão (ã)":"Irmão (ã)",
    "Ex-combatente (o próprio)":"Ex-combatente (o próprio)",
    "Pessoa designada (Beneficiário instituído)":"Pessoa designada (Beneficiário instituído)",
    "Não informado" : "Não informado",
    "Outros (Pessoas sem Vínculo Militar)" : "Outros (Pessoas sem Vínculo Militar)"
    }


In [55]:
def categorizando(x):
    if x in pensoes:
        return pensoes[x]


In [56]:
df['tipo_pensao_agrupado'] = df['tipo_pensao'].apply(categorizando)

In [59]:
df= df[['ano', 'mes', 'tipo_pensao','tipo_pensao_agrupado', 'total']]
df

,ano,mes,tipo_pensao,tipo_pensao_agrupado,total
0,2024,4,Não informado,Não informado,1
1,2024,4,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
2,2024,4,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
3,2024,4,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
4,2024,4,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
...,...,...,...,...,...
1162588,2020,4,Filha,"Filhos, netos, enteados ou menores em tutela",1
1162589,2020,4,Filha,"Filhos, netos, enteados ou menores em tutela",1
1162590,2020,4,Filha,"Filhos, netos, enteados ou menores em tutela",1
1162591,2020,4,Filha,"Filhos, netos, enteados ou menores em tutela",1


In [70]:
df['tipo_pensao_agrupado'].unique()

array(['Não informado', 'Cônjuge ou ex-cônjuge',
       'Filhos, netos, enteados ou menores em tutela', 'Pais',
       'Irmão (ã)', 'Pessoa designada (Beneficiário instituído)',
       'Ex-combatente (o próprio)',
       'Outros (Pessoas sem Vínculo Militar)'], dtype=object)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162593 entries, 0 to 1162592
Data columns (total 5 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   ano                   1162593 non-null  Int64 
 1   mes                   1162593 non-null  Int64 
 2   tipo_pensao           1162593 non-null  object
 3   tipo_pensao_agrupado  1162593 non-null  object
 4   total                 1162593 non-null  Int64 
dtypes: Int64(3), object(2)
memory usage: 47.7+ MB


In [62]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [64]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes','INTEGER',description='Mês de referência da observação'),        
        bigquery.SchemaField('tipo_pensao','STRING',description='Tipo de pensão'),
        bigquery.SchemaField('tipo_pensao_agrupado','STRING',description='Categorização das pensões'),
        bigquery.SchemaField('total','INTEGER',description='Quantidade total da observação')
        ]

In [67]:
table_ref = dataset_ref.table('MILITARES_pensionistas_total') 
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=cca767c1-083a-4141-8a3f-f27423760375>